In [2]:

from azure.ai.ml import MLClient, command, Output
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential


# Connect to AML workspace
ml_client = MLClient.from_config(DefaultAzureCredential())


# Define environment
donut_env = Environment(
    name="donut-lora-env",
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-2.0-cuda11.7:latest",
    conda_file="environment.yaml"
)

# Register environment
ml_client.environments.create_or_update(donut_env)


job = command(
    code="./src",
    command="python train.py --data_dir ./data --output_dir ${{outputs.model_output}}",
    environment=donut_env,
    compute="anishswiss1",
    display_name="donut-lora-train",
    experiment_name="donut-lora-exp",
    outputs={
        "model_output": Output(type="uri_folder", mode="upload")
    },
)


# Submit
returned_job = ml_client.jobs.create_or_update(job)
print(f"Submitted job: {returned_job.name}")




Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Submitted job: tidy_iron_bn39qzkpdd


In [16]:
%pip install azure-ai-ml azure-identity transformers torch pillow

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Initialize MLClient
ml_client = MLClient.from_config(DefaultAzureCredential())

# Download registered model locally
download_path = "./donut_qa_model"
ml_client.models.download(
    name="donutQA",  # Change to your model name
    version="1",                      # Change to your model version (optional if you want latest)
    download_path=download_path
)



Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'https://test4762514297.blob.core.windows.net/azureml/ExperimentRun/dcid.tidy_iron_bn39qzkpdd/outputs' './donut_qa_model/donutQA/outputs' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


OSError: [Errno 9] Bad file descriptor: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/anishswiss1/code/Users/anishswiss/DonutQA/donut_qa_model/donutQA/outputs/donut-lora/model.safetensors'

In [1]:
%pip install torch torchvision torchaudio


from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import os

download_path = "./donut_qa_model"


model_folder = os.path.join(download_path, "donutQA/outputs/donut-lora")  # may need adjustment

files_only = [f for f in os.listdir(model_folder) if os.path.isfile(os.path.join(model_folder, f))]
print("FILES .... ")
print(files_only)

processor = DonutProcessor.from_pretrained(model_folder)
model = VisionEncoderDecoderModel.from_pretrained(model_folder)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
FILES .... 
['.amlignore', '.amlignore.amltmp', 'added_tokens.json', 'config.json', 'generation_config.json', 'model.safetensors', 'preprocessor_config.json', 'sentencepiece.bpe.model', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json']


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
image = Image.open("test_pay_stub.jpg").convert("RGB")
question = "What is the net pay?"

prompt = f"<s_docvqa><s_question>{question}</s_question><s_answer>"


In [6]:
# Prepare inputs
pixel_values = processor(image, return_tensors="pt").pixel_values
decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids

# Generate prediction
outputs = model.generate(
    pixel_values,
    decoder_input_ids=decoder_input_ids,
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

# Decode answer
sequence = processor.batch_decode(outputs.sequences)[0]
answer = sequence.split("<s_answer>")[1].split("</s_answer>")[0]

print(f"Predicted answer: {answer}")


Predicted answer:  $853.30
